In [ ]:
import geopandas as gpd  # برای خواندن و تحلیل داده‌های مکانی (شکل‌فایل‌ها)
import numpy as np       # برای کار با آرایه‌ها و محاسبات عددی
from shapely.geometry import Point  # برای کار با هندسه‌های نقطه‌ای
from scipy.spatial.distance import cdist  # برای محاسبه فاصله بین نقاط (خانه‌ها و مدارس)
import random  # برای تولید اعداد تصادفی

# ---------- ۱. خواندن فایل‌های مکانی ----------
houses = gpd.read_file(r'C:\Users\lenovo\Desktop\data\houses.shp')  # خواندن فایل خانه‌ها
schools = gpd.read_file(r'C:\Users\lenovo\Desktop\data\schools.shp')  # خواندن فایل مدارس

houses_centroids = houses.geometry.centroid  # گرفتن مرکز هندسی (centroid) هر خانه
schools_centroids = schools.geometry.centroid  # گرفتن مرکز هندسی هر مدرسه

house_coords = np.array([[p.x, p.y] for p in houses_centroids])  # استخراج مختصات X,Y خانه‌ها
school_coords = np.array([[p.x, p.y] for p in schools_centroids])  # استخراج مختصات X,Y مدارس

distance_matrix = cdist(house_coords, school_coords)  # ماتریس فاصله بین هر خانه با هر مدرسه

num_houses = distance_matrix.shape[0]  # تعداد کل خانه‌ها (ردیف‌های ماتریس)
num_schools = distance_matrix.shape[1]  # تعداد مدارس (ستون‌های ماتریس)
capacity = 38  # ظرفیت هر مدرسه (حداکثر ۳۸ خانه)

# ---------- ۲. تعریف تابع هدف (Fitness Function) ----------
def fitness(solution):
    school_counts = [0] * num_schools  # شمارش خانه‌های تخصیص‌ یافته به هر مدرسه
    total_distance = 0  # مجموع فاصله خانه‌ها تا مدارس

    for i, school in enumerate(solution):  # پیمایش همه خانه‌ها
        total_distance += distance_matrix[i, school]  # جمع کردن فاصله آن خانه با مدرسه انتخابی
        school_counts[school] += 1  # افزایش تعداد خانه‌های اختصاص‌یافته به آن مدرسه

    penalty = sum((max(0, count - capacity) ** 2 for count in school_counts))  # جریمه مدارس بیش‌از‌ظرفیت
    return total_distance + penalty * 10000  # مجموع فاصله + جریمه (ضرب در عدد بزرگ برای تأثیر زیاد)

# ---------- ۳. تولید زنبور (راه‌حل تصادفی معتبر) ----------
def generate_bee():
    counts = [0] * num_schools  # شمارنده خانه‌ها برای هر مدرسه
    allocation = []  # لیست نهایی تخصیص

    for _ in range(num_houses):  # برای هر خانه
        choices = [i for i in range(num_schools) if counts[i] < capacity]  # مدارسی که هنوز جا دارند
        school = random.choice(choices)  # انتخاب یک مدرسه تصادفی از بین مدارس آزاد
        allocation.append(school)  # افزودن مدرسه انتخابی به لیست تخصیص
        counts[school] += 1  # افزایش شمارش برای آن مدرسه

    return allocation  # بازگرداندن تخصیص نهایی

# ---------- ۴. جستجوی محلی اطراف یک زنبور ----------
def local_search(bee):
    new_bee = bee[:]  # کپی از زنبور اصلی
    i = random.randint(0, num_houses - 1)  # انتخاب یک خانه تصادفی

    # مدارسی که ظرفیت دارند یا همان مدرسه‌ای که در حال حاضر به آن اختصاص دارد
    valid_choices = [j for j in range(num_schools) if new_bee.count(j) < capacity or new_bee[i] == j]

    if valid_choices:
        new_bee[i] = random.choice(valid_choices)  # تغییر مدرسه آن خانه به یکی از مدارس مجاز

    return new_bee  # بازگرداندن زنبور جدید (تغییر یافته)

# ---------- ۵. اجرای الگوریتم زنبور ----------
num_bees = 30  # تعداد کل زنبورها
num_elite = 5  # تعداد ناحیه‌های نخبه (زنبورهای برتر)
num_local_search = 10  # تعداد جستجوی محلی برای هر زنبور نخبه
iterations = 100  # تعداد تکرار الگوریتم

bees = [generate_bee() for _ in range(num_bees)]  # تولید جمعیت اولیه زنبورها

best_solution = None  # بهترین پاسخ تا کنون
best_fitness = float('inf')  # مقدار اولیه شایستگی (خیلی زیاد)

for iteration in range(iterations):  # حلقه اصلی الگوریتم
    fitnesses = [fitness(b) for b in bees]  # ارزیابی تمام زنبورها
    sorted_indices = np.argsort(fitnesses)  # مرتب‌سازی شاخص‌ها بر اساس مقدار تابع هدف (از کم به زیاد)
    bees = [bees[i] for i in sorted_indices]  # مرتب‌سازی زنبورها بر اساس شایستگی

    # بروزرسانی بهترین جواب
    if fitnesses[sorted_indices[0]] < best_fitness:
        best_fitness = fitnesses[sorted_indices[0]]
        best_solution = bees[0]

    new_bees = []  # لیست زنبورهای جدید

    # جستجوی محلی برای زنبورهای نخبه
    for i in range(num_elite):
        for _ in range(num_local_search):
            new_bees.append(local_search(bees[i]))

    # تولید زنبورهای تصادفی (کاوشگر)
    while len(new_bees) < num_bees:
        new_bees.append(generate_bee())

    bees = new_bees  # جمعیت جدید جایگزین جمعیت قبلی می‌شود

# ---------- ۶. نمایش خروجی نهایی ----------
print(f"\n✅ بهترین مقدار تابع هدف (فاصله + جریمه): {best_fitness:.1f}")  # مقدار نهایی بهترین راه‌حل
print("📌 تعداد خانه تخصیص‌داده‌شده به هر مدرسه:")
for i in range(num_schools):
    print(f"مدرسه {i+1}: {best_solution.count(i)} خانه")  # شمارش خانه‌های اختصاص‌یافته به هر مدرسه

print("📍 خانه اول → مدرسه‌ها:", best_solution[:20])  # نمایش ۲۰ خانه اول و مدرسه اختصاص‌داده‌شده



✅ بهترین مقدار تابع هدف (فاصله + جریمه): 308706.5
📌 تعداد خانه تخصیص‌داده‌شده به هر مدرسه:
مدرسه 1: 38 خانه
مدرسه 2: 38 خانه
مدرسه 3: 38 خانه
مدرسه 4: 38 خانه
مدرسه 5: 38 خانه
مدرسه 6: 38 خانه
مدرسه 7: 38 خانه
مدرسه 8: 38 خانه
مدرسه 9: 38 خانه
مدرسه 10: 38 خانه
📍 خانه اول → مدرسه‌ها: [0, 6, 5, 9, 3, 5, 9, 2, 0, 6, 0, 9, 8, 4, 6, 7, 4, 8, 5, 8]


## توضیحات کامل پروژه

در این تمرین، مسئله‌ی اختصاص بهینه‌ی ۳۸۰ خانه به ۱۰ مدرسه بررسی شده است. هدف اصلی، کاهش مجموع فاصله‌ی بین خانه‌ها و مدارسی است که به آن‌ها اختصاص داده می‌شوند، به‌گونه‌ای که ظرفیت هر مدرسه از ۳۸ خانه تجاوز نکند. برای حل این مسئله از الگوریتم زنبور مصنوعی استفاده شده است که یکی از روش‌های هوش ازدحامی برای بهینه‌سازی مسائل پیچیده است.

ابتدا داده‌های مکانی مربوط به خانه‌ها و مدارس از فایل‌های شکلی (فایل‌های مکانی) خوانده شده‌اند و مختصات هندسی مراکز هر خانه و مدرسه به‌دست آمده‌اند. سپس ماتریس فاصله‌ی بین هر خانه و هر مدرسه بر اساس فاصله‌ی اقلیدسی محاسبه شده است.

الگوریتم زنبور مصنوعی شامل مراحل زیر است:
۱. تولید جمعیت اولیه از زنبورها (راه‌حل‌های تصادفی ولی معتبر، بدون نقض ظرفیت).
۲. ارزیابی هر زنبور با استفاده از تابع هدف (مجموع فاصله به علاوه‌ی جریمه برای مدارس بیش‌از‌ظرفیت).
۳. مرتب‌سازی زنبورها براساس مقدار تابع هدف و انتخاب زنبورهای نخبه.
۴. اجرای جستجوی محلی در اطراف زنبورهای نخبه برای بهبود راه‌حل‌ها.
۵. ایجاد زنبورهای جدید کاوشگر (با راه‌حل‌های کاملاً جدید) برای حفظ تنوع.
۶. تکرار فرایند برای تعداد مشخصی از تکرارها و در نهایت انتخاب بهترین پاسخ.

در پایان، بهترین مقدار تابع هدف و نحوه‌ی توزیع خانه‌ها بین مدارس به‌صورت آماری و نمونه‌ای نمایش داده شده‌اند. این الگوریتم موفق شد تمام خانه‌ها را به‌طور دقیق و بدون تخطی از ظرفیت، به مدارس اختصاص دهد.
